In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Using TensorFlow backend.


In [2]:
#define classifier 
classifier = Sequential()

In [3]:
#step 1 Convolution
#input shape (256, 256, 3) => means 3 channels = RGB, 256 pixel x 256 pixel
#input shape (256, 256, 3) => means 2 channels = Grayscale, 256 pixel x 256 pixel
#input shape can be compressed to 3 or 256 x 256
#number of filters equals to featured convoluted map produce (32)
#kernel_size : size of convolution filter = (3,3) means 3 x 3
#strides = the distance the filter convoluted, default value is (1,1)
#activation function to get the nonlinearity = 'relu'
classifier.add(Conv2D(filters=32, 
                      kernel_size = (3, 3), 
                      input_shape=(64, 64, 3), 
                      activation='relu'))

In [4]:
#step 2 Pooling
#pool_size by default (2,2)
#strides default is None if None it will be set as like pool_size
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [5]:
#to optimize we can have 2nd convolution layer
#we don't need to declare the input map since it has been set by previous layer
classifier.add(Conv2D(filters=32, 
                      kernel_size = (3, 3), 
                      activation='relu'))

In [6]:
#add 2nd pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [7]:
#step 3 flatenning
classifier.add(Flatten())

In [8]:
#step 4 Full Connection
#units (nodes in the layer should not be too small) = by experimentation is 128
classifier.add(Dense(units = 128, 
                     activation='relu'))

In [9]:
#output layer => output layer only 1 node because our dependent variable 
#is binary outcome cat or dog
classifier.add(Dense(units = 1, 
                     activation='sigmoid'))

In [10]:
#optimizer = 'adam' is the stohastic gradient descent
#loss function => binary_crossentropy for 2 outcomes, if we have 3 outcomes
# we should use categorical_crossentropy
classifier.compile(optimizer="adam", 
                   loss = "binary_crossentropy",
                   metrics = ["accuracy"])

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

#target_size is the dimension expected by the CNN 64 x 64 
#batch_size is the number of samples will be included in the sets that will
# go through the CNN which the weights will be updated
#class_mode should be set it is binary or more than 2 categories 
training_set = train_datagen.flow_from_directory(
        '../Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        '../Section 40 - Convolutional Neural Networks (CNN)/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
#epoch is the number of training set of each epochs (8000 images will be train each epoch)
#validation_steps => corresponds to the number of images in our test set
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000,
        epochs=2,
        validation_data=test_set,
        validation_steps=2000)

Epoch 1/2
8000/8000 [==============================] - 1323s 165ms/step - loss: 0.0330 - acc: 0.9888 - val_loss: 1.3560 - val_acc: 0.7780
Epoch 2/2
1714/8000 [=====>........................] - ETA: 14:11 - loss: 0.0292 - acc: 0.9900